In [3]:
import random
import pandas as pd
import PySimpleGUI as sg
from collections import Counter
import operator
sg.theme('DarkGrey15')

file_path = './Lottery_Powerball_Winning_Numbers__Beginning_2010.csv' # Load CSV file
data_df = pd.read_csv(file_path)
data_df['Draw Date'] = pd.to_datetime(data_df['Draw Date']) #
data_df.sort_values('Draw Date', ascending=False, inplace=True) # Sort by date from newest to oldest
sorted_dates = data_df['Draw Date'].dt.strftime('%m/%d/%Y').tolist() # Convert dates back to string format for display in GUI

# Ball probability calculator
def analyze_data():
    # Convert the second column of CSV data to a list
    locationWinningNumbers = data_df.iloc[:, 1].tolist() #

    # Separate the first five numbers and the last number of each row
    first_five_numbers = [sublist.split()[:5] for sublist in locationWinningNumbers]
    last_number = [sublist.split()[-1] for sublist in locationWinningNumbers]

    # Split numbers into individual elements
    split_first_five = [int(num) for sublist in first_five_numbers for num in sublist]
    split_last_number = [int(num) for num in last_number]

    # Calculate the frequency of each number
    first_five_counter = Counter(split_first_five)
    last_number_counter = Counter(split_last_number)

    # Calculate total counts
    total_first_five = sum(first_five_counter.values())
    total_last_number = sum(last_number_counter.values())

    # Calculate the probability of occurrence for each number
    first_five_probabilities = {num: count / total_first_five for num, count in first_five_counter.items()}
    last_number_probabilities = {num: count / total_last_number for num, count in last_number_counter.items()}
    return first_five_probabilities, last_number_probabilities

# (1) "Random number" button event
def button_click1():
    random_balls = generate_numbers()  # Your random number generation function
    sg.Popup('Random Number', random_balls)

def generate_numbers():
    whiteBalls = random.sample(range(1, 70), 5) # First five numbers, range 1 to 69
    redball = random.randint(1, 26) # Sixth number, range 1 to 26
    return f"The white balls: {whiteBalls}\nThe red balls: [{redball}]" 

# (2) "Analyze input numbers" button event
def button_click2():
   winningNumberProbability = analyze_numbers(window)  # Function to analyze numbers
   sg.Popup('The probability of occurrence', winningNumberProbability)

def analyze_numbers(window): #
    first_five_probabilities, sixth_probabilities = analyze_data()
    user_numbers = []    
    for i in range(1, 7):
        value = window[f'-IN{i}-'].get() # Retrieve the user's input numbers from PySimpleGUI's input fields

        if value:  # Check if the input is not empty
            user_numbers.append(int(value))
        else:
            user_numbers.append(0)  # Use a default value for empty inputs

    # Check if the user's input numbers are within the correct range
    if not all(1 <= num <= 69 for num in user_numbers[:5]) or not (1 <= user_numbers[5] <= 26):
        return 'Invalid input!\nThe first five numbers should be in the range 1-69 and the sixth number should be in the range 1-26.'
    # Check if the numbers in user_numbers[:5] are all unique
    if len(set(user_numbers[:5])) < 5:
        return 'Invalid input!\nThe first five numbers must all be unique.' 

    user_first_five = user_numbers[:5]
    user_sixth = user_numbers[5]

    probabilities = [first_five_probabilities.get(num, 0) for num in user_first_five]
    probabilities.append(sixth_probabilities.get(user_sixth, 0))
   
    prob1, prob2, prob3, prob4, prob5, prob6 = probabilities

    return f"Your number:{user_numbers[0]}, probability of occurrence(%)：{round(prob1*100, 3)} \nYour number:{user_numbers[1]}, probability of occurrence(%)：{round(prob2*100, 3)} \nYour number:{user_numbers[2]}, probability of occurrence(%)：{round(prob3*100, 3)} \nYour number:{user_numbers[3]}, probability of occurrence(%)：{round(prob4*100, 3)} \nYour number:{user_numbers[4]}, probability of occurrence(%)：{round(prob5*100, 3)} \nYour number:{user_numbers[5]}, probability of occurrence(%)：{round(prob6*100, 3)}"

# (3) select_draw function
def select_draw(selected_draw=None):
    winningNumbersSeries = data_df[data_df["Draw Date"] == selected_draw]["Winning Numbers"]
    if not winningNumbersSeries.empty: # Check if the series is not empty
        winningNumbers = winningNumbersSeries.iloc[0] # Get the first element of the series
        return f"Winning Numbers on {selected_draw} is：{winningNumbers}"
    else:
        return "No Records。"
# (3) Show The Previs Wining Number
def button_Show_The_Previs_Wining_Number( ):  
   oldWinningNumber = select_draw(window['-COMBO-'].get()  ) # Get the selected date from the combo box
   sg.Popup('The Previs Wining Number', oldWinningNumber)


# (4) Show Top 10 most frequent first five numbers and their probabilities
def button4_top_5_Red_ball():
    first_five_probabilities, last_number_probabilities = analyze_data()

    # Find the top 10 numbers
    top_10_first_five = sorted(first_five_probabilities.items(), key=operator.itemgetter(1), reverse=True)[:10]

    # Convert the top 10 numbers to a string
    top_10_str = ','.join(f'{num}: {round(prob*100, 3)}%' for num, prob in top_10_first_five)
    
    # Split the string into multiple parts
    FirstFive_parts = top_10_str.split(',')
    
    # Join the parts into a single string
    FirstFive = '\n'.join(FirstFive_parts)

    sg.Popup('Top 10 most frequent white balls and their probabilities:\n', FirstFive)

# (5) Top 5 most frequent red ball and their probabilities
def button5_top_5_Powerball():
    first_five_probabilities, last_number_probabilities = analyze_data()

    # Find the top 5 numbers
    top_5_last_number = sorted(last_number_probabilities.items(), key=operator.itemgetter(1), reverse=True)[:5]

    # Convert the top 5 numbers to a string
    top_5_str = ','.join(f'{num}: {round(prob*100, 3)}%' for num, prob in top_5_last_number)

    # Split the string into multiple parts
    winningball_parts = top_5_str.split(',')
    
    # Join the parts into a single string
    top_5_winningball = '\n'.join(winningball_parts)
    sg.Popup('Top 5 most frequent red ball and their probabilities:\n', top_5_winningball)




# Initialize PySimpleGUI window
layout = [
    [sg.Column([    
    [sg.Text("(1) Computer Number Selector", size=(40,1))],
    [sg.Button("Generate Random Numbers", size=(40,1), key='-BUTTON1-', bind_return_key=True)],

    [sg.Text("(2) Check Your Number", size=(40,1))],  
    [sg.Text("Enter Numbers", size=(10,1)), 
     sg.Input(size=(4,5), key='-IN1-',text_color="white"), 
     sg.Input(size=(4,5), key='-IN2-',text_color="white"), 
     sg.Input(size=(4,5), key='-IN3-',text_color="white"), 
     sg.Input(size=(4,5), key='-IN4-',text_color="white"), 
     sg.Input(size=(4,5), key='-IN5-',text_color="white"), 
     sg.Input(size=(3,6), key='-IN6-',text_color="red"),],
    [sg.Button("Analyze My Numbers", size=(40,1), key='-BUTTON2-', bind_return_key=True)],
    [sg.Button('Reset', size=(40,1))],
     
    [sg.Text("(3) Previous Results", size=(40,1))], 
    [sg.Text("Select Past Draw Date:", size=(20,1)), sg.Combo(sorted_dates, size=(20,10), key='-COMBO-', enable_events=True)],#enable_events=True
    [sg.Button("Show Previous Results", size=(40,1), key='-BUTTON3-', bind_return_key=True)],#bind_return_key=True

    [sg.Text("(4) Data Analysis - Popular WINNING NUMBERS", size=(40,1))],
    [sg.Button('Show Top 10 White Balls', size=(40,1))],

    [sg.Text("(5) Data Analysis - Popular Powerball", size=(40,1))],
    [sg.Button('Show Top 5 Red Ball', size=(40,1))],

    [sg.Button('Close Program', size=(40,1))]

     ])]]


# Create the window
window = sg.Window("Lottery Helper",  layout, size=(375, 425))

# Create an event loop
while True:
    event, values = window.read()
    # End program if user closes window or     # presses the OK button
    if event == sg.WINDOW_CLOSED or event == 'Close Program':
        break
    elif event == '-BUTTON1-':
        button_click1()
    elif event == '-BUTTON2-':
        button_click2()
    elif event == '-BUTTON3-':
        button_Show_The_Previs_Wining_Number()
    elif event == 'Show Top 10 White Balls':
        button4_top_5_Red_ball()    
    elif event == 'Show Top 5 Red Ball':
        button5_top_5_Powerball()            
    elif event == 'Reset':
        # Clear the input fields
        for i in range(1, 7):
            window[f'-IN{i}-'].update('')    
window.close()





